<a href="https://www.kaggle.com/code/kaiyoo88/tutorial-chatbot-rag-langchain?scriptVersionId=205319807" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
pip install -q langchain langchain-community langchain_huggingface chromadb wikipedia

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 31.0.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.

In [4]:
import os
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [30]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR-API-TOKEN"

In [31]:
os.environ.get("HUGGINGFACEHUB_API_TOKEN")

'YOUR-API-TOKEN'

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

# set Korean embedding and llm odel
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

hf_llm = HuggingFaceHub(
    repo_id="skt/kogpt2-base-v2",
    model_kwargs={"task": "text-generation"} ## question-answering tasK X. text-generation
)

/tmp/ipykernel_30/3726747796.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_30/3726747796.py:7: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  hf_llm = HuggingFaceHub(


In [8]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup

# By default, English documents (https://en.wikipedia.org))
# def load_Wiki_docs(query):
#     loader = WikipediaLoader(query=query, load_max_docs=1)
#     documents = loader.load()
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200
#     )
#     splits = text_splitter.split_documents(documents)
    
#     return splits


# For Korean query, get results from Korean wikipedia website and crawl and parse results
def load_Korean_wiki_docs(topic):

    url = f"https://ko.wikipedia.org/wiki/{topic}"
    
    response = requests.get(url)
    response.raise_for_status()  # raise Exception when error occurs

    # HTML parsing and extract body contents
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.find('div', {'class': 'mw-parser-output'})  # find div including body contents 
    
    # Extract contents
    paragraphs = content.find_all('p')
    text = "\n".join([p.get_text() for p in paragraphs])  # concat all context in <p> tags 
 
    # convert to Document object (required for LangChain)
    documents = [Document(page_content=text, metadata={"source": url})]
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

In [10]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
    return vectorstore

In [11]:
topic = "흑백요리사"
# Load wikipedia documents for this topic
splits = load_Korean_wiki_docs(topic)
# Create vectorstore with this fetched docs
vectorstore = create_vectorstore(splits)

In [12]:
splits

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《호날두요리사》였다.[1]')]

## 1. Use langchain RAG

In [14]:
def create_rag_chain(vectorstore):
    prompt_template = """문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: {context}
    질문: {question}
    답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

    # Make context shorter
    # def short_context(context, max_length=300):
    #     return context[:max_length] if len(context) > max_length else context
    
    # class ShortContextRetriever(BaseRetriever):
    #     def __init__(self, retriever):
    #         super().__init__()
    #         self._retriever = retriever
        
    #     def get_relevant_documents(self, query):
    #         docs = self._retriever.get_relevant_documents(query)
    #         for doc in docs:
    #             doc.page_content = short_context(doc.page_content)
    #         return docs
    
    # retriever = ShortContextRetriever(vectorstore.as_retriever())

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [15]:
# create langchang RAG chain
qa_chain = create_rag_chain(vectorstore)

In [16]:
question = "심사위원을 누가 맡았어?"

# result = qa_chain({"query": question})
result = qa_chain.invoke({"query": question})

print ("결과:")
print(result["result"])

print("출처:")
for doc in result["source_documents"]:
    print(doc.page_content)
    print("---")

결과:
문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《호날두요리사》였다.[1]
    질문: 심사위원을 누가 맡았어?
    답변: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원: 쉐프들이 심사위원으로 참여했어?
( 쉐프들이 심사위원으로 참여했어?)
심사위원:
출처:
《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《호날두요리사》였다.[1]
---


In [18]:
docs = vectorstore.as_retriever().get_relevant_documents(question)
docs

/tmp/ipykernel_30/2742417880.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vectorstore.as_retriever().get_relevant_documents(question)


[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《호날두요리사》였다.[1]')]

In [20]:
docs = vectorstore.similarity_search(question, k=4)
docs

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《호날두요리사》였다.[1]')]

In [ ]:
# It seems vectorDB loading from embedding model works fine, but seems llm model does not.
# Some Korean llm model seems to work fine in text-generation task, but for Question-Ansering task, we might need another approach.

## 2. Use QA pipeline with vectorstor similarity search

In [21]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "yjgwak/klue-bert-base-finetuned-squard-kor-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set Q_A pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/246k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [22]:
# Example: define question and context 
question = "오늘 날씨 어때?"
context = "오늘의 날씨는 맑고 따뜻한 기온이 유지될 것으로 보입니다."

# model chain
result = qa_pipeline(question=question, context=context)

# Result
print("질문:", question)
print("답변:", result['answer'])

질문: 오늘 날씨 어때?
답변: 맑고 따뜻한 기온이


In [28]:
# search context in VectorStore
def retrieve_context(question, vectorstore):
    docs = vectorstore.similarity_search(question, k=4)
    if docs:
        return " ".join([doc.page_content for doc in docs])
        # return docs[0].page_content  # return first relevant doc
    else:
        return None

# Generate answer based on query and searched context similar to RAG chain
def answer_question_with_context(question, vectorstore):
    context = retrieve_context(question, vectorstore)
    if context:
        result = qa_pipeline(question=question, context=context)
        return result['answer'], context  # return answer and used source doc
    else:
        return "관련 문맥을 찾지 못했습니다.", None

In [29]:
# Example
question = "심사위원을 누가 맡았어?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)
print("사용된 문맥:", used_context)

질문: 심사위원을 누가 맡았어?
답변: 백종원과 안성재가
사용된 문맥: 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《호날두요리사》였다.[1]
